In [1]:
# 03_time_series.ipynb

%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import logging
from datetime import datetime
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
from prophet import Prophet

from setup_utils import load_intermediate_data, plr, save_intermediate_data

# For demonstration, you can either:
# - Use the original full dataset and re-derive 'df' (like in unsupervised step),
#   or load the intermediate data and filter as needed.
df = load_intermediate_data('data/intermediate_unsupervised.csv')

# Now run your time-series models (ARIMA, Prophet)
# Below is a simplified snippet just to show the structure.

# Let's say we forecast 'heart_rate' for a single subject
subject_id = 10020187
metric = 'heart_rate'
subject_df = df[df['subject_id'] == subject_id].copy()
subject_df['charttime'] = pd.to_datetime(subject_df['charttime'])

# Ensure proper frequency
subject_df = subject_df.sort_values('charttime')
subject_df.set_index('charttime', inplace=True)
subject_daily = subject_df[metric].resample('D').mean().interpolate()

# Fit an ARIMA model as an example
arima_model = auto_arima(subject_daily, seasonal=False, stepwise=True, trace=False, error_action='ignore')
future_periods = 30
forecast_arima = arima_model.predict(n_periods=future_periods)

# Convert forecast to a DataFrame
future_dates = pd.date_range(start=subject_daily.index.max()+pd.Timedelta('1D'), periods=future_periods)
forecast_df = pd.DataFrame({'charttime': future_dates, 'forecast': forecast_arima})
forecast_df['subject_id'] = subject_id
forecast_df['metric'] = metric

print(forecast_df.head())
plr()

# Prophet Example (if desired):
prophet_df = subject_daily.reset_index().rename(columns={'charttime': 'ds', metric:'y'})
m = Prophet()
m.fit(prophet_df)
future = m.make_future_dataframe(periods=future_periods)
prophet_forecast = m.predict(future)
print(prophet_forecast[['ds','yhat','yhat_lower','yhat_upper']].head())

# (Optional) Save time-series results
save_intermediate_data(forecast_df, 'data/intermediate_timeseries.csv')



Importing plotly failed. Interactive plots will not work.
C:\Users\steve\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\steve\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\steve\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\steve\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\steve\AppData\Local\P

            charttime   forecast  subject_id      metric
2170-02-26 2170-02-26  65.313516    10020187  heart_rate
2170-02-27 2170-02-27  69.613420    10020187  heart_rate
2170-02-28 2170-02-28  70.596193    10020187  heart_rate
2170-03-01 2170-03-01  59.815961    10020187  heart_rate
2170-03-02 2170-03-02  67.336577    10020187  heart_rate
@24/12/12 05:48:20


05:48:20 - cmdstanpy - INFO - Chain [1] done processing


          ds       yhat  yhat_lower  yhat_upper
0 2169-01-15  66.417058   65.642647   67.246397
1 2169-01-16  66.330306   65.538349   67.117169
2 2169-01-17  66.250709   65.444194   67.063842
3 2169-01-18  66.258933   65.476617   67.047850
4 2169-01-19  66.355855   65.542606   67.128419
